# Градиентный бустинг на решающих деревьях

## Как правильно перебирать параметры

В этом ноутбуке мы будем заниматься поиском оптимальных параметров для градиентного бустинга, перебирая задания по заданной сетке. В этом задании от Вас не будет требоваться найти самые лучшие параметры, но всё равно важно правильно составлять сетку для перебора. Для этого нужно понимать суть параметров и их смысл.



**learning_rate** -- темп обучения нашего метода. Для этого метода сетка перебора должна быть логарифмической, т.е. перебирать порядковые значения (к примеру, [1e-3, 1e-2, 1e-1, 1]). В большинстве случаев достаточно перебрать значения от 1e-5 до 1.<br />
**max_depth** -- максимальная глубина деревьев в ансамбле. Вообще говоря, эта величина зависит от числа признаков, но обычно лучше растить небольшие деревья. К примеру, библиотека CatBoost, которую мы будем исследовать сегодня, рекомендует перебирать значения до 10 (и уточняется, что обычно оптимальная глубина лежит от 6 до 10).<br />
**n_estimators** -- количество деревьев в ансамбле. Обычно стоит перебирать с каким-то крупным шагом (можно по логарифмической сетке). Здесь важно найти баланс между производительностью, временем обучения и качеством. Обычно нескольких тысяч деревьев бывает достаточно.<br />

Учтите, что в реальных задачах необходимо следить за тем, что оптимальные значения параметров не попадают на границы интервалов, т.е. что вы нашли хотя бы локальный минимум. Если Вы перебрали значения параметра от 1 до 10 и оказалось, что 10 - оптимальное значение, значит следует перебрать и бОльшие числа, чтобы убедиться, что качество не улучшается дальше (или по крайней мере убедиться, что рост качества сильно замедляется и на сильное улучшения рассчитывать не стоит.


## Подготовка датасета

Все библиотеки, используемые сегодня, мы будем проверять на одних и тех же параметрах: n_estimators=1000, max_depth=5, learning_rate=0.1. Таким образом мы устанавливаем, соответственно, число деревьев в ансамбле равным 1000, ограничиваем максимальную глубину деревьев 5 и устанавливаем темп обучения равным 0.1.

In [43]:
%matplotlib inline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.metrics import mean_squared_error
from hyperopt import hp, tpe, Trials
from hyperopt.fmin import fmin
from hyperopt.pyll import scope
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time

In [44]:
test_parameters = {"n_estimators": 1000, "max_depth": 5, "learning_rate":0.1}

df = pd.read_csv('dataframe_YesIndex_YesHeader_C.csv', index_col=0)
df.head()

,Engine Capacity,Cylinders,Drive Type,Fuel Tank Capacity,Fuel Economy,Fuel Type,Horsepower,Torque,Transmission,Top Speed,...,Acceleration,Length,Width,Height,Wheelbase,Trunk Capacity,name,price,currency,Country
0,1.2,3,0,42.0,4.9,0,76,100.0,0,170,...,14.0,4.245,1.670,1.515,2.550,450.0,Mitsubishi Attrage 2021 1.2 GLX (Base),34099.0,0,0
1,1.2,3,0,42.0,4.9,0,76,100.0,0,170,...,14.0,4.245,1.670,1.515,2.550,450.0,Mitsubishi Attrage 2021 1.2 GLX (Base),34099.0,0,0
2,1.4,4,0,45.0,6.3,0,75,118.0,1,156,...,16.0,3.864,1.716,1.721,2.513,2800.0,Fiat Fiorino 2021 1.4L Standard,41250.0,0,0
3,1.6,4,0,50.0,6.4,0,102,145.0,0,180,...,11.0,4.354,1.994,1.529,2.635,510.0,Renault Symbol 2021 1.6L PE,44930.0,0,0
4,1.5,4,0,48.0,5.8,0,112,150.0,0,170,...,10.9,4.314,1.809,1.624,2.585,448.0,MG ZS 2021 1.5L STD,57787.0,0,0


В глаза бросаются две проблемы: у нас имеется информация о производителе автомобиля, скрытая в текстовой переменной модели автомобиля (нет отдельного признака). Также имеются некоторые категориальные переменные.

**Данные**: датасет со стоимостью поддержанных автомобилей  
**Цели**: В данном задании следует выполнить следующие пункты (выполнять можно в любом порядке)
1. Изучить датасет, проверить наличие пропусков. При необходимости заменить их на среднее значение признака.
3. Добавить столбец brand с информацией о производителе автомобиля (для простоты можно взять первое слово в названии модели). Столбец name удалить из датасета
4. Решить, какие признаки Вы хотите сделать категориальными. Конвертировать выбранные категориальные столбцы в тип category. 
5. Создать датасет А с категориальными признаками в виде категорий. Для этого необходимо создать вектор целевых значений (столбец цен автомобилей) и матрицу признаков с категориальными переменными в виде категорий (получается путем удаления только целевой переменной из матрицы с данными). Дополнительно стоит создать список с названиями и индексами столбцов категориальных переменных (поможет в будущем).
6. Создать датасет B, с удаленными категориальными признаками.
7. Создать датасет C с категориальными признаками в виде one-hot encoding. Для этого необходимо создать вектор целевых значений (столбец цен автомобилей), удалить из матрицы признаков столбец с целевыми переменными и все категориальными переменные, а затем добавить новые признаки, соответствующие one-hot encoding категориальных переменных (здесь вам поможет функция `pd.get_dummies`).
8. Разбить датасеты на тренировочное и тестовое множества, используя `train_test_split(X, y, test_size=0.25, random_state=0)`

In [45]:
datasets = {'A' : None, 'B': None, 'C': None}

**Работа с датасетом**

In [46]:
# Проверяем наличие пропусков
print("Пропуски в данных:")
print(df.isnull().sum())
# if df.isnull().sum().any(): # замена пропусков средним значением признака, если они есть
#     df.fillna(df.mean(), inplace=True)

Пропуски в данных:
Engine Capacity       0
Cylinders             0
Drive Type            0
Fuel Tank Capacity    0
Fuel Economy          0
Fuel Type             0
Horsepower            0
Torque                0
Transmission          0
Top Speed             0
Seating Capacity      0
Acceleration          0
Length                0
Width                 0
Height                0
Wheelbase             0
Trunk Capacity        0
name                  0
price                 0
currency              0
Country               0
dtype: int64


**Добавляем столбец brand**

In [47]:
df['brand'] = df['name'].str.split().str[0] 
df.drop(columns=['name'], inplace=True) 
df.head()

,Engine Capacity,Cylinders,Drive Type,Fuel Tank Capacity,Fuel Economy,Fuel Type,Horsepower,Torque,Transmission,Top Speed,...,Acceleration,Length,Width,Height,Wheelbase,Trunk Capacity,price,currency,Country,brand
0,1.2,3,0,42.0,4.9,0,76,100.0,0,170,...,14.0,4.245,1.670,1.515,2.550,450.0,34099.0,0,0,Mitsubishi
1,1.2,3,0,42.0,4.9,0,76,100.0,0,170,...,14.0,4.245,1.670,1.515,2.550,450.0,34099.0,0,0,Mitsubishi
2,1.4,4,0,45.0,6.3,0,75,118.0,1,156,...,16.0,3.864,1.716,1.721,2.513,2800.0,41250.0,0,0,Fiat
3,1.6,4,0,50.0,6.4,0,102,145.0,0,180,...,11.0,4.354,1.994,1.529,2.635,510.0,44930.0,0,0,Renault
4,1.5,4,0,48.0,5.8,0,112,150.0,0,170,...,10.9,4.314,1.809,1.624,2.585,448.0,57787.0,0,0,MG


**Конвертация признаков в тип category**

In [48]:
categorical_columns = ['Transmission', 'Seating Capacity', 'Cylinders', 'Country', 'brand'] # категориальные признаки
df[categorical_columns] = df[categorical_columns].astype('category') # конвертирую в тип category

**Создаем датасет А с категориальными признаками в виде категорий**

In [49]:
A = df.copy()

categorical_columns_A = ['Transmission', 'Seating Capacity', 'Cylinders', 'Country', 'brand'] # категориальные признаки

y = A['price'] # вектор целевых значений

A = A.drop(columns=['price']) # удаление столбца 'Price' из матрицы признаков

categorical_columns_ind_A = [A.columns.get_loc(col) for col in categorical_columns_A] # список с названиями и индексами столбцов категориальных переменных для датасета А

A[categorical_columns_A] = A[categorical_columns_A].astype('category') # преобразуем категориальные признаки в тип 'category' для датасета А

print(categorical_columns_ind_A)

[8, 10, 1, 18, 19]


**Создаем датасет B, с удаленными категориальными признаками**

In [50]:
B = df.copy()

categorical_columns_B = ['Transmission', 'Seating Capacity', 'Cylinders', 'Country', 'brand']

B = B.drop(columns=categorical_columns_B)

print(B.head())

   Engine Capacity  Drive Type  Fuel Tank Capacity  Fuel Economy  Fuel Type  \
0              1.2           0                42.0           4.9          0   
1              1.2           0                42.0           4.9          0   
2              1.4           0                45.0           6.3          0   
3              1.6           0                50.0           6.4          0   
4              1.5           0                48.0           5.8          0   

   Horsepower  Torque  Top Speed  Acceleration  Length  Width  Height  \
0          76   100.0        170          14.0   4.245  1.670   1.515   
1          76   100.0        170          14.0   4.245  1.670   1.515   
2          75   118.0        156          16.0   3.864  1.716   1.721   
3         102   145.0        180          11.0   4.354  1.994   1.529   
4         112   150.0        170          10.9   4.314  1.809   1.624   

   Wheelbase  Trunk Capacity    price  currency  
0      2.550           450.0  34099.

**Создать датасет C с категориальными признаками в виде one-hot encoding**

In [51]:
C = df.copy()

y_C = C['price'] # Вектор целевых значений (столбец цен автомобилей)

X_C1 = C.drop(columns=['price', 'Transmission', 'Seating Capacity', 'Cylinders', 'Country', 'brand'])

X_C_get_dum = pd.get_dummies(['Transmission', 'Seating Capacity', 'Cylinders', 'Country', 'brand'])

C = pd.concat([X_C1, X_C_get_dum], axis=1)
C = C.astype('category')
print(C.head())

  Engine Capacity Drive Type Fuel Tank Capacity Fuel Economy Fuel Type  \
0             1.2          0               42.0          4.9         0   
1             1.2          0               42.0          4.9         0   
2             1.4          0               45.0          6.3         0   
3             1.6          0               50.0          6.4         0   
4             1.5          0               48.0          5.8         0   

  Horsepower Torque Top Speed Acceleration Length  Width Height Wheelbase  \
0         76  100.0       170         14.0  4.245  1.670  1.515     2.550   
1         76  100.0       170         14.0  4.245  1.670  1.515     2.550   
2         75  118.0       156         16.0  3.864  1.716  1.721     2.513   
3        102  145.0       180         11.0  4.354  1.994  1.529     2.635   
4        112  150.0       170         10.9  4.314  1.809  1.624     2.585   

  Trunk Capacity currency Country Cylinders Seating Capacity Transmission  \
0          450.

**Разбваем датасеты на тренировочное и тестовое множества**

In [52]:
X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(A, y, test_size=0.25, random_state=0)

X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(B, y, test_size=0.25, random_state=0)

X_train_C, X_test_C, y_train_C, y_test_C = train_test_split(C, y_C, test_size=0.25, random_state=0)

**Задания**:
1. Обучите любую понравившуюся вам модель градиентного бустинга (CatBoost, XGBoost, LightGBM) для предсказания стоимости автомобиля на всех построенных датасетах (A, B и C)
2. Подберите оптимальный набор параметров модели с помощью библиотеки hyperopt

**Обучение XGBRegressor для предсказания стоимости автомобиля на всех построенных датасетах (A, B и C)**

In [53]:
# Обучение и оценка модели на датасете A
model_A = XGBRegressor(enable_categorical=True)
model_A.fit(X_train_A, y_train_A)
score_A = model_A.score(X_test_A, y_test_A)
print("Score on dataset A:", score_A)

# Обучение и оценка модели на датасете B
model_B = XGBRegressor(enable_categorical=True)
model_B.fit(X_train_B, y_train_B)
score_B = model_B.score(X_test_B, y_test_B)
print("Score on dataset B:", score_B)

# Обучение и оценка модели на датасете C
model_C = XGBRegressor(enable_categorical=True)
model_C.fit(X_train_C, y_train_C)
score_C = model_C.score(X_test_C, y_test_C)
print("Score on dataset C:", score_C)

Score on dataset A: 0.9700250418619975
Score on dataset B: 0.9977233646103306
Score on dataset C: 0.9191948523167089


**Подбор оптимального набора параметров модели с помощью библиотеки hyperopt**

In [64]:
from hyperopt import hp, tpe, Trials, STATUS_OK, fmin
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

def hyperopt_objective(params): # Определение функции для оптимизации
    model = XGBRegressor(**params, enable_categorical=True, verbosity=0) # Оценка модели с помощью кросс-валидации
    scores = cross_val_score(model, X_train_A, y_train_A, cv=5, scoring='r2') # Возвращаем результат для оптимизации (отрицательное среднее значение R^2 для максимизации)

    return {'loss': -scores.mean(), 'status': STATUS_OK}

space = {
    'n_estimators': hp.choice('n_estimators', range(100, 1500, 100)),
    'max_depth': hp.choice('max_depth', range(1, 11)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.5),
    'gamma': hp.uniform('gamma', 0, 20),
    'min_child_weight': hp.uniform('min_child_weight', 0, 10),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
}

trials = Trials() # Запуск поиска оптимальных параметров
best_params = fmin(hyperopt_objective, space, algo=tpe.suggest, max_evals=1000, trials=trials)

print("Best parameters:", best_params)


100%|████████████████████████████████████████████| 1000/1000 [52:08<00:00,  3.13s/trial, best loss: -0.701000429385455]
Best parameters: {'colsample_bytree': 0.9801044480038991, 'gamma': 2.7533654608482836, 'learning_rate': 0.010394199670235972, 'max_depth': 9, 'min_child_weight': 9.59881126178149, 'n_estimators': 2, 'subsample': 0.5112689938277549}


In [65]:
best_n_estimators = 300
best_max_depth = 10
best_learning_rate = best_params['learning_rate']
best_gamma = best_params['gamma']
best_min_child_weight = best_params['min_child_weight']
best_subsample = best_params['subsample']
best_colsample_bytree = best_params['colsample_bytree']



# Создание моделей с лучшими параметрами
best_model_A = XGBRegressor(n_estimators=best_n_estimators,
                            max_depth=best_max_depth,
                            learning_rate=best_learning_rate,
                            gamma=best_gamma,
                            min_child_weight=best_min_child_weight,
                            subsample=best_subsample,
                            colsample_bytree=best_colsample_bytree,
                            enable_categorical=True)
best_model_A.fit(X_train_A, y_train_A)
best_score_A = best_model_A.score(X_test_A, y_test_A)
print("Best score on dataset А:", best_score_A)


best_model_B = XGBRegressor(n_estimators=best_n_estimators,
                            max_depth=best_max_depth,
                            learning_rate=best_learning_rate,
                            gamma=best_gamma,
                            min_child_weight=best_min_child_weight,
                            subsample=best_subsample,
                            colsample_bytree=best_colsample_bytree,
                            enable_categorical=True)
best_model_B.fit(X_train_B, y_train_B)
best_score_B = best_model_B.score(X_test_B, y_test_B)
print("Best score on dataset B:", best_score_B)


best_model_C = XGBRegressor(n_estimators=best_n_estimators,
                            max_depth=best_max_depth,
                            learning_rate=best_learning_rate,
                            gamma=best_gamma,
                            min_child_weight=best_min_child_weight,
                            subsample=best_subsample,
                            colsample_bytree=best_colsample_bytree,
                            enable_categorical=True)

best_model_C.fit(X_train_C, y_train_C)
best_score_C = best_model_C.score(X_test_C, y_test_C)
print("Best score on dataset C:", best_score_C)

Best score on dataset А: 0.8948836191171743
Best score on dataset B: 0.9626154554209462
Best score on dataset C: 0.8817447295283086
